In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os
import glob
import node2vec

c:\Users\konst\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
path = "Datasets/"

In [3]:
def getProtectedAttributesTwitch(G, df):
    protected_attributes = dict(zip(df["numeric_id"], df["mature"]))
    newG = nx.set_node_attributes(G, protected_attributes, "mature")
    return newG

def getProtectedAttributesDeezer(G, df):
    protected_attributes = dict(zip(df["id"], df["target"]))
    newG = nx.set_node_attributes(G, protected_attributes, "gender")
    return newG

def getProtectedAttributesPokec(G,df):
    protected_attributes = dict(zip(df[0], df[3]))
    newG = nx.set_node_attributes(G, protected_attributes, "gender")
    return newG

def getprotectedAttributesDict_Facebook(featuresDF,featureNameDF,egoFeatDF):
    gender_index = featureNameDF.index[featureNameDF[1] == "gender;anonymized"].to_list()[0]
    featuresDF = featuresDF[[0, gender_index + 1]]
    egoFeatDF = egoFeatDF[[0, gender_index + 1]]
    featuresDict = dict(zip(featuresDF[0], featuresDF[gender_index + 1]))
    egoFeatDict = dict(zip(egoFeatDF[0], egoFeatDF[gender_index + 1]))
    featuresDict.update(egoFeatDict)
    return featuresDict

def getEgoFeats(path):
    featFiles = []
    featNameFiles = []
    egoFeatFiles = []
    for file in os.listdir(path):
        if file.endswith(".feat"):
            featFiles.append(file)
        if file.endswith(".featnames"):
            featNameFiles.append(file)
        if file.endswith(".egofeat"):
            egoFeatFiles.append(file)
    return featFiles, featNameFiles, egoFeatFiles

def getProtectedAttributesFacebook(G):
    node_gender_dict = {}
    localpath = "Datasets/facebook/facebook/"
    fbFeatFiles, fbFeatNameFiles, fbEgoFeatFiles = getEgoFeats(localpath)
    for index in range(len(fbFeatFiles)):
        localFeaturesDF = pd.read_csv(localpath + fbFeatFiles[index], sep=" ", header=None)
        localFeatureNamesDf = pd.read_csv(localpath + fbFeatNameFiles[index], sep=" ", header=None)
        localEgoFeatDf = pd.read_csv(localpath + fbEgoFeatFiles[index], sep=" ", header=None)
        protectedAttrDict = getprotectedAttributesDict_Facebook(localFeaturesDF, localFeatureNamesDf, localEgoFeatDf)
        node_gender_dict.update(protectedAttrDict)
    
    newG = nx.set_node_attributes(G, node_gender_dict, "gender")
    return newG


In [4]:
# Twitch gamers Graph
twitchGamers_graph = nx.read_edgelist("{}twitch_gamers/large_twitch_edges.csv".format(path), nodetype=int, delimiter=",")

In [5]:
# Twitch gamers features
twitchGamers_features = pd.read_csv("{}twitch_gamers/large_twitch_features.csv".format(path))

In [6]:
getProtectedAttributesTwitch(twitchGamers_graph, twitchGamers_features)

In [7]:
# Deezer europe graph
deezer_graph = nx.read_edgelist("{}deezer_europe/deezer_europe/deezer_europe_edges.csv".format(path), nodetype=int, delimiter=",")

In [8]:
# Read Deezer genders
deezer_gendersDf = pd.read_csv("{}/deezer_europe/deezer_europe/deezer_europe_target.csv".format(path))

In [9]:
getProtectedAttributesDeezer(deezer_graph, deezer_gendersDf)

In [10]:
pokec_graph = nx.read_edgelist("{}/pokec/soc-pokec-relationships.txt".format(path), nodetype=int, delimiter="\t", create_using=nx.DiGraph())

In [11]:
pokec_features = pd.read_csv("{}/pokec/soc-pokec-profiles.txt".format(path), delimiter="\t", header=None, usecols=[0,3])

In [12]:
getProtectedAttributesPokec(pokec_graph, pokec_features)

In [13]:
facebook_graph_all = nx.read_edgelist("{}facebook/facebook/facebook_combined.txt".format(path), nodetype=int, delimiter=" ")

In [14]:
getProtectedAttributesFacebook(facebook_graph_all)